In [1]:
from dotenv import load_dotenv
from langchain_fireworks import ChatFireworks
from langchain_core.output_parsers import StrOutputParser

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

load_dotenv()
llm = ChatFireworks(model="accounts/fireworks/models/llama-v3-70b-instruct",temperature=0)
system_message_prompt = SystemMessagePromptTemplate.from_template("你是一个求职助手，用汉语交流。")
human_message_prompt = HumanMessagePromptTemplate.from_template("HR问或说：“{user_input}”，你用汉语回答：")
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt,
     human_message_prompt]
)
chain = chat_prompt|llm|StrOutputParser()
text = """
请问，最近有换工作的意愿么？我们正在寻找一位团队伙伴。
"""
messages = chat_prompt.format_prompt(user_input=text)
print(chain.invoke(messages))



"嗨，感谢您的邀请！我目前确实有换工作的意愿，想寻找一个挑战性的角色和良好的团队文化。我对加入您的团队感到非常兴奋，能否了解更多关于这个岗位的信息？"


In [2]:
# 完整的求职检索链
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from dotenv import load_dotenv
from langchain_fireworks import FireworksEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.runnables import RunnableLambda

load_dotenv()
# 文档加载、分割
loader = WebBaseLoader("https://raw.githubusercontent.com/baiziyuandyufei/langchain-self-study-tutorial/main/jl.txt")
raw_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    add_start_index=False,
)
documents = text_splitter.split_documents(raw_documents)
print(f"分割后快数: {len(documents)}")
# 向量化、存储
embedding_model = FireworksEmbeddings(model="nomic-ai/nomic-embed-text-v1.5")
db = FAISS.from_documents(documents=documents, embedding=embedding_model)
print(f"索引片段数: {db.index.ntotal}")
# 检索器
retriever = db.as_retriever()
# 检索链 返回检索结果字符串
question_retrieval_chain = retriever | RunnableLambda(lambda docs: "\n".join([doc.page_content for doc in docs]))

USER_AGENT environment variable not set, consider setting it to identify your requests.


分割后快数: 140
索引片段数: 140


In [3]:
print(question_retrieval_chain.invoke("文本分类"))

长文本的预测。语种识别，汉英分词与关键词抽取。受模型对输入文本长度的限制，也是为了确保模型预测准确性，对于所有输入文本，加入预处理步骤，执行语种识别，分词和关键词抽取，将抽取出的关键词送入模型进行预
•	商品标题分类项目
]
}
•	社交短文本分类项目
1. 数据的获取
•	社交短文本实体识别项目


In [4]:
# 完整的问题分类链
from langchain_fireworks import ChatFireworks
from dotenv import load_dotenv
from langchain_core.runnables import RunnableLambda
import re
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
# 实例化聊天模型
chat = ChatFireworks(model="accounts/fireworks/models/llama-v3-70b-instruct", 
                    temperature=0.3,
                    top_p=0.3)
# 分类词典
question_classify_dict = {
    "离职原因": {
        "response": "有换工意愿，上家公司离我居住地太远，通勤时间太长。",
        "examples": [{"text": "离职/换工作的原因","label": "离职原因"}]
    },
    "薪资": {
        "response": "我期望薪资为30K～40K。",
        "examples": [{"text": "但是我们应该最高30K，一般还达不到.","label": "薪资"}]
    },
    "外包&外协&外派&驻场": {
        "response": "请发送或说明职位的办公地点定位。以及薪资范围。我期望薪资范围30-40K？",
        "examples": [{"text": "你好，我们是外协岗位，在国家电网 南瑞工作的","label": "外包&外协&外派&驻场"}]
    },
    "兼职": {
        "response": "职位的办公地点在哪？薪资多少，怎么结算？",
        "examples": [{"text": "哈喽～本职位为线上兼职，一单一结款，根据自己时间自由接单，不耽误自己的主业，您看感兴趣嘛？","label":"兼职"}]
    },
    "预约面试": {
        "response": "本周内上午、下午都有时间。",
        "examples": [{"text": "想约您面试，方便的话麻烦告诉我一下您可以约面试的日期及时间【请选择工作日内的上午10-12点或下午14点到17点内的时间】。","label":"预约面试"}]
    },
    "到岗时间": {
        "response": "两周内到岗。",
        "examples": [{"text": "咱到岗时间呢。","label":"到岗时间"}]
    },
    "其他": {
        "response": "",
        "examples": []
    }
}

# 准备分类提示所需信息
def prepare_question_classify_prompt():
    # [问题分类链] 构造问题分类示例集
    examples = []
    for key in question_classify_dict:
        r_examples = question_classify_dict[key]["examples"]
        if len(r_examples) > 0:
            examples.extend(r_examples)
    # [问题分类链] 分类示例模板 作用就是格式化分类示例
    example_prompt = PromptTemplate.from_template(
        """文本: {text}
        类别: {label}
        """
    )
    # [问题分类链] 写在所有示例之前的分类要求
    prefix = f"""
    给出每个文本的类别，类别只能属于以下列出的一种

    {"- ".join(question_classify_dict.keys())}

    如果不属于以上类别，则类别名称为“其他”。

    例如：
    """
    # [问题分类链] 写在所有示例之后的内容，提示LLM对当前input进行分类
    suffix = """文本: {input}\n类别:
    """
    return examples,example_prompt,prefix,suffix

# 构建分类提示
examples,example_prompt,prefix,suffix = prepare_question_classify_prompt()
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"],
    example_separator="\n"
)

# 由分类标签生成分类响应
def label_to_response(label):
    label = re.sub('类别: ?', '', label)
    label = label if label in question_classify_dict else "其他"
    response = question_classify_dict[label]["response"]
    return response

# 分类链 返回分类响应字符串
question_classify_chain = few_shot_prompt | chat | StrOutputParser()|RunnableLambda(label_to_response)
question_classify_chain.invoke("何时到岗？")


/Users/feiyu/miniconda3/envs/langchain-py11/lib/python3.11/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


'两周内到岗。'

In [5]:
# 合并分类链检索链构建完整链
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.prompts import SystemMessagePromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 系统提示
system_message_prompt = SystemMessagePromptTemplate.from_template(f"""
你是求职助手于先生。刚从中国科学院信息工程研究所离职。居住地为北京。
""")
# 人工提示
human_message_prompt = HumanMessagePromptTemplate.from_template("""
HR问或说: {question}。

{context}

请用汉语回复内容，内容的头部和尾部不要出现引号。
""")
# 聊天提示
prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, 
     human_message_prompt])

# 求最长公共子串
def longest_common_substring(s1, s2):
    # 获取两个字符串的长度
    len_s1 = len(s1)
    len_s2 = len(s2)

    # 创建一个二维数组用来存储动态规划的结果
    dp = [[0] * (len_s2 + 1) for _ in range(len_s1 + 1)]

    # 初始化最大长度和结束位置
    max_length = 0
    end_pos = 0

    # 填充动态规划表
    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                if dp[i][j] > max_length:
                    max_length = dp[i][j]
                    end_pos = i
            else:
                dp[i][j] = 0

    # 提取最大公共子串
    start_pos = end_pos - max_length
    return s1[start_pos:end_pos]

# 合并分类和问答提示为context提示
def generate_context_prompt(all_dict):
    question = all_dict["question"]
    question_classify_response = all_dict["context"]["question_classify_response"]
    question_retrieval_response = all_dict["context"]["question_retrieval_response"]
    
    if len(question_classify_response) > 0:
        question_classify_template = f"""你在回答中体现以下内容
        {question_classify_response}
        """
    else:
        question_classify_template = ""
    
    if len(longest_common_substring(question,question_retrieval_response)) >=4:
        question_retrieval_template = f"""工作经历有以下内容
        {question_retrieval_response}
        """
    else:
        question_retrieval_template = f"""没有针对该问题的简历信息。
        """
    return {
        "question":question,
        "context":f"{question_classify_template}\n\n{question_retrieval_template}\n\n"
    }

# 整体链
final_chain = {"question": RunnablePassthrough(),
               "context":  RunnableParallel(question_classify_response=question_classify_chain,
                                            question_retrieval_response=question_retrieval_chain) 
} | RunnableLambda(generate_context_prompt)| prompt | chat | StrOutputParser() 

print(final_chain.invoke("介绍一下word weaver"))

Word Weaver是一款自然语言处理工具，主要用于文本生成和编辑。它可以根据用户的输入，生成高质量的文本，例如文章、邮件、报告等。Word Weaver使用了先进的算法和机器学习技术，能够理解用户的意图和需求，生成符合要求的文本。

在我的研究生涯中，我曾经参与过自然语言处理相关的项目，了解到Word Weaver的原理和应用场景。虽然我没有直接使用过Word Weaver，但我认为它是一款非常有用的工具，能够提高文本生成和编辑的效率和质量。

如果贵公司需要自然语言处理方面的技术支持，我愿意提供帮助和建议。
